<!--BOOK_INFORMATION-->
<img align="left" style="width:80px;height:98px;padding-right:20px;" src="https://raw.githubusercontent.com/joe-papa/pytorch-book/main/files/pytorch-book-cover.jpg">

This notebook contains an excerpt from the [PyTorch Pocket Reference](http://pytorchbook.com) book by [Joe Papa](http://joepapa.ai); content is available [on GitHub](https://github.com/joe-papa/pytorch-book).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joe-papa/pytorch-book/blob/main/06_05_Pruning.ipynb)

# Chapter 6 - PyTorch Acceleration & Optimization

### Pruning

In [ ]:
import torch 
from torch import nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(
            F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(
            F.relu(self.conv2(x)), 2)
        x = x.view(-1, 
                   int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
device = torch.device("cuda" if 
    torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)

print(list(model.conv1.named_parameters()))
# out: 
# [('weight', Parameter containing:
# tensor([[[[ 0.0560,  0.0066, ...0.0183,  0.0783]]]], 
#        device='cuda:0',
#        requires_grad=True)), 
#  ('bias', Parameter containing:
# tensor([ 0.0754, -0.0356,  ... , -0.0111,  0.0984], 
#        device='cuda:0',
#        requires_grad=True))]           

In [ ]:
import torch.nn.utils.prune as prune

prune.random_unstructured(model.conv1, 
                          name="weight", 
                          amount=0.25)

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

In [ ]:
prune.random_unstructured(model.conv1, 
                          name="bias", 
                          amount=0.25)

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

In [ ]:
model = LeNet5().to(device)

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d): 
        prune.random_unstructured(module, 
                              name='weight', 
                              amount=0.3) # <1>
    elif isinstance(module, torch.nn.Linear):
        prune.random_unstructured(module, 
                              name='weight', 
                              amount=0.5) # <2>

In [ ]:
model = LeNet5().to(device)

parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight'),
    (model.fc2, 'weight'),
    (model.fc3, 'weight'),
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.25)

In [ ]:
class MyPruningMethod(prune.BasePruningMethod):
  PRUNING_TYPE = 'unstructured'

  def compute_mask(self, t, default_mask):
    mask = default_mask.clone()
    mask.view(-1)[::2] = 0
    return mask

def my_unstructured(module, name):
  MyPruningMethod.apply(module, name)
  return module

In [ ]:
model = LeNet5().to(device)
my_unstructured(model.fc1, name='bias')

Linear(in_features=400, out_features=120, bias=True)